# Silver para Gold

## Arquitetura Medallion - Camada Gold

Este script realiza a transformação dos dados da camada Silver para a camada Gold.

### Operações realizadas:
- Agregação de dados em métricas de negócio
- Criação de tabelas fato e dimensão (modelo estrela)
- Preparação de datasets analíticos otimizados para consultas
- Derivação de KPIs e indicadores chave

### Regras da camada Gold:
- Dados agregados e prontos para análise
- Esquema otimizado para consultas analíticas
- Suporte a dashboards e relatórios
- Foco na facilidade de uso para analistas

# Importação de bibliotecas

In [1]:
import pandas as pd
import numpy as np
import os
import sqlite3
from datetime import datetime, timedelta
import json

# Configuração de Diretórios

In [2]:
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), '..'))
SILVER_DIR = os.path.join(BASE_DIR, 'silver')    
GOLD_DIR = os.path.join(BASE_DIR, 'gold')        

if not os.path.exists(GOLD_DIR):
    os.makedirs(GOLD_DIR)

# Conexão com o banco de dados Silver

In [3]:
silver_db_path = os.path.join(SILVER_DIR, 'silver_layer.db')
silver_conn = sqlite3.connect(silver_db_path)

gold_db_path = os.path.join(GOLD_DIR, 'gold_layer.db')
gold_conn = sqlite3.connect(gold_db_path)

tables_query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql(tables_query, silver_conn)['name'].tolist()

print(f"Tabelas disponíveis na camada Silver: {tables}")

Tabelas disponíveis na camada Silver: ['customers', 'orders', 'products', 'order_items', 'order_payments', 'order_reviews', 'sellers']


# Funções auxiliares para transformações Gold

In [4]:
def extract_date_dimensions(df, date_column):
    """Extrai dimensões de data a partir de uma coluna de data."""
    if date_column not in df.columns or not pd.api.types.is_datetime64_dtype(df[date_column]):
        print(f"Erro: {date_column} não é uma coluna de data válida.")
        return df
    
    df[f'{date_column}_year'] = df[date_column].dt.year
    df[f'{date_column}_month'] = df[date_column].dt.month
    df[f'{date_column}_day'] = df[date_column].dt.day
    df[f'{date_column}_weekday'] = df[date_column].dt.weekday
    df[f'{date_column}_quarter'] = df[date_column].dt.quarter
    df[f'{date_column}_week'] = df[date_column].dt.isocalendar().week
    
    return df

def create_time_dimension(start_date, end_date):
    """Cria uma tabela de dimensão de tempo."""
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    
    time_dim = pd.DataFrame({
        'date_id': date_range,
        'date': date_range,
        'year': date_range.year,
        'quarter': date_range.quarter,
        'month': date_range.month,
        'month_name': date_range.strftime('%B'),
        'week': date_range.isocalendar().week,
        'day': date_range.day,
        'weekday': date_range.weekday,
        'weekday_name': date_range.strftime('%A'),
        'is_weekend': date_range.weekday.isin([5, 6]),  
    })
    
    time_dim['fiscal_quarter'] = time_dim['month'].apply(lambda m: ((m-1)//3) + 1)
    time_dim['season'] = time_dim['month'].apply(lambda m: 
                                           'Summer' if m in [6, 7, 8] else
                                           'Fall' if m in [9, 10, 11] else
                                           'Winter' if m in [12, 1, 2] else 'Spring')
    
    return time_dim

# Carregar tabelas necessárias da camada Silver

In [5]:
orders_df = pd.read_sql("SELECT * FROM orders", silver_conn)
order_items_df = pd.read_sql("SELECT * FROM order_items", silver_conn)
products_df = pd.read_sql("SELECT * FROM products", silver_conn)
customers_df = pd.read_sql("SELECT * FROM customers", silver_conn)
sellers_df = pd.read_sql("SELECT * FROM sellers", silver_conn)
order_payments_df = pd.read_sql("SELECT * FROM order_payments", silver_conn)
reviews_df = pd.read_sql("SELECT * FROM order_reviews", silver_conn)

# Converter colunas de data para datetime

In [6]:
date_columns = {
    'orders': [col for col in orders_df.columns if 'date' in col],
    'order_items': [col for col in order_items_df.columns if 'date' in col],
    'reviews': [col for col in reviews_df.columns if 'date' in col]
}

for col in date_columns['orders']:
    orders_df[col] = pd.to_datetime(orders_df[col], errors='coerce')

for col in date_columns['order_items']:
    order_items_df[col] = pd.to_datetime(order_items_df[col], errors='coerce')

for col in date_columns['reviews']:
    reviews_df[col] = pd.to_datetime(reviews_df[col], errors='coerce')

# 1. Criação de Dimensão de Tempo

In [7]:
print("Tipos antes da conversão:")
for col in date_columns['orders']:
    print(f"orders_df['{col}']: {orders_df[col].dtype}")
for col in date_columns['order_items']:
    print(f"order_items_df['{col}']: {order_items_df[col].dtype}")

orders_df['order_purchase_timestamp'] = pd.to_datetime(orders_df['order_purchase_timestamp'], errors='coerce')
order_items_df['shipping_limit_date'] = pd.to_datetime(order_items_df['shipping_limit_date'], errors='coerce')

min_date = min(
    orders_df['order_purchase_timestamp'].min(),
    order_items_df['shipping_limit_date'].min()
)
max_date = max(
    orders_df['order_delivered_customer_date'].max(),
    orders_df['order_estimated_delivery_date'].max()
)

print(f"Data mínima: {min_date}")
print(f"Data mínima: {min_date}")
print(f"Data máxima: {max_date}")

time_dim = create_time_dimension(min_date, max_date)

time_dim.to_sql('dim_time', gold_conn, if_exists='replace', index=False)
print(f"Criada dimensão de tempo: {len(time_dim)} linhas")

Tipos antes da conversão:
orders_df['order_delivered_carrier_date']: datetime64[ns]
orders_df['order_delivered_customer_date']: datetime64[ns]
orders_df['order_estimated_delivery_date']: datetime64[ns]
order_items_df['shipping_limit_date']: datetime64[ns]
Data mínima: 2016-09-04 21:15:19
Data mínima: 2016-09-04 21:15:19
Data máxima: 2018-11-12 00:00:00
Criada dimensão de tempo: 799 linhas


# 2. Criação de Dimensão de Produtos

In [8]:
dim_products = products_df.copy()

dim_products['product_volume_cm3'] = dim_products['product_length_cm'] * \
                                    dim_products['product_height_cm'] * \
                                    dim_products['product_width_cm']
dim_products['product_is_heavy'] = dim_products['product_weight_g'] > 3000

dim_products.to_sql('dim_products', gold_conn, if_exists='replace', index=False)
print(f"Criada dimensão de produtos: {len(dim_products)} linhas")

Criada dimensão de produtos: 32951 linhas


# 3. Criação de Dimensão de Clientes

In [9]:
dim_customers = customers_df.copy()

dim_customers.to_sql('dim_customers', gold_conn, if_exists='replace', index=False)

print(f"Criada dimensão de clientes: {len(dim_customers)} linhas")

Criada dimensão de clientes: 99441 linhas


# 4. Criação de Dimensão de Vendedores

In [10]:
dim_sellers = sellers_df.copy()

dim_sellers.to_sql('dim_sellers', gold_conn, if_exists='replace', index=False)

print(f"Criada dimensão de vendedores: {len(dim_sellers)} linhas")

Criada dimensão de vendedores: 3095 linhas


# 5. Criação da Tabela Fato de Vendas

In [11]:
fact_sales = pd.merge(order_items_df, orders_df, on='order_id', how='left')
fact_sales = pd.merge(fact_sales, order_payments_df, on='order_id', how='left')
fact_sales = pd.merge(fact_sales, customers_df[['customer_id', 'customer_unique_id']], on='customer_id', how='left')

fact_sales = extract_date_dimensions(fact_sales, 'order_purchase_timestamp')
fact_sales = extract_date_dimensions(fact_sales, 'order_delivered_customer_date')

fact_sales.to_sql('fact_sales', gold_conn, if_exists='replace', index=False)
print(f"Criada tabela fato de vendas: {len(fact_sales)} linhas")

Criada tabela fato de vendas: 117604 linhas


# 6. Criação da Tabela Fato de Avaliações

In [12]:
fact_reviews = reviews_df.copy()

fact_reviews = pd.merge(fact_reviews, orders_df[['order_id', 'customer_id']], on='order_id', how='left')

fact_reviews = extract_date_dimensions(fact_reviews, 'review_creation_date')

fact_reviews.to_sql('fact_reviews', gold_conn, if_exists='replace', index=False)
print(f"Criada tabela fato de avaliações: {len(fact_reviews)} linhas")

Criada tabela fato de avaliações: 99224 linhas


# 7. Criação de Agregações para Analytics

# 7.1. Agregação de vendas por categoria de produto

In [13]:
sales_by_category = fact_sales.merge(
    dim_products[['product_id', 'product_category_name', 'product_category_name_english']],
    on='product_id', how='left'
)

sales_by_category_agg = sales_by_category.groupby('product_category_name_english').agg({
    'order_id': 'count',
    'price': 'sum',
    'freight_value': 'sum'
}).reset_index()

sales_by_category_agg.columns = ['category', 'num_orders', 'total_revenue', 'total_freight']
sales_by_category_agg['avg_ticket'] = sales_by_category_agg['total_revenue'] / sales_by_category_agg['num_orders']

sales_by_category_agg.to_sql('agg_sales_by_category', gold_conn, if_exists='replace', index=False)
print(f"Criada agregação de vendas por categoria: {len(sales_by_category_agg)} linhas")

Criada agregação de vendas por categoria: 72 linhas


# 7.2. Agregação de vendas por tempo

In [14]:
sales_by_time = fact_sales.groupby(['order_purchase_timestamp_year', 'order_purchase_timestamp_month']).agg({
    'order_id': pd.Series.nunique,
    'price': 'sum',
    'customer_unique_id': pd.Series.nunique
}).reset_index()

sales_by_time.columns = ['year', 'month', 'num_orders', 'total_revenue', 'num_customers']
sales_by_time['avg_ticket'] = sales_by_time['total_revenue'] / sales_by_time['num_orders']
sales_by_time['revenue_per_customer'] = sales_by_time['total_revenue'] / sales_by_time['num_customers']

sales_by_time.to_sql('agg_sales_by_time', gold_conn, if_exists='replace', index=False)
print(f"Criada agregação de vendas por tempo: {len(sales_by_time)} linhas")

Criada agregação de vendas por tempo: 24 linhas


# 7.3. Agregação de avaliações por categoria

In [15]:
reviews_by_category = fact_reviews.merge(
    fact_sales[['order_id', 'product_id']], on='order_id', how='left'
).merge(
    dim_products[['product_id', 'product_category_name_english']], on='product_id', how='left'
)

reviews_by_category_agg = reviews_by_category.groupby('product_category_name_english').agg({
    'review_id': 'count',
    'review_score': 'mean'
}).reset_index()

reviews_by_category_agg.columns = ['category', 'num_reviews', 'avg_score']
reviews_by_category_agg.to_sql('agg_reviews_by_category', gold_conn, if_exists='replace', index=False)
print(f"Criada agregação de avaliações por categoria: {len(reviews_by_category_agg)} linhas")

Criada agregação de avaliações por categoria: 72 linhas


# 7.4. KPIs gerais de vendas

In [16]:
total_orders = len(orders_df)
total_revenue = fact_sales['price'].sum()
avg_ticket = total_revenue / total_orders
avg_delivery_time = (orders_df['order_delivered_customer_date'] - orders_df['order_purchase_timestamp']).mean().days

kpis = pd.DataFrame({
    'metric': ['total_orders', 'total_revenue', 'avg_ticket', 'avg_delivery_time_days'],
    'value': [total_orders, total_revenue, avg_ticket, avg_delivery_time]
}, index=None)

kpis.to_sql('kpis_sales', gold_conn, if_exists='replace', index=False)
print("Criados KPIs gerais de vendas")

silver_conn.close()
gold_conn.close()

print("\nProcessamento para camada Gold concluído!")

Criados KPIs gerais de vendas

Processamento para camada Gold concluído!


# Salvar metadados da camada Gold

In [17]:
gold_tables = [
    'dim_time', 'dim_products', 'dim_customers', 'dim_sellers',
    'fact_sales', 'fact_reviews',
    'agg_sales_by_category', 'agg_sales_by_time', 'agg_reviews_by_category',
    'kpis_sales'
]

gold_metadata = {
    'tables': gold_tables,
    'processing_date': datetime.now().isoformat(),
    'description': 'Camada Gold - Tabelas dimensionais e fatos para análise de negócio'
}

with open(os.path.join(GOLD_DIR, 'gold_metadata.json'), 'w') as f:
    json.dump(gold_metadata, f, indent=2)

print("Metadados salvos com sucesso!")

Metadados salvos com sucesso!
